# Generate UFC Graphs
This file will gather the UFC fight data from a google docs link, clean the data, and then generate
the graphs with netwrorkx

In [10]:
import pandas as pd
import numpy as np
import networkx as nx
import csv

In [11]:
url = 'https://drive.google.com/file/d/1FtCv5uUaBY_GHzFrBmqpwJ5EhoVNnY4U/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
data = pd.read_csv(url)
len(data)

6012

# Clean the data
The first step here is to remove rows from nan values in the stat columns.

We'll still be using the complete data for now.

In [26]:
# Remove na values in the B fighter data (this covers all of it, i checked with tableau)
cleaned_data = data.dropna(subset=['B_avg_KD','B_Reach_cms','B_Height_cms','B_Stance','B_age'])
len(cleaned_data)

4173

In [27]:
# Remove na values in the R fighter data (this covers all of it, i checked with tableau)
cleaned_data = cleaned_data.dropna(subset=['R_avg_KD','R_Reach_cms'])
len(cleaned_data)

3915

# Generate Undirected Graph

In [13]:
'''The following variable assignments are to isolate all unique fighters into one list 
(which we can later use to get our nodes)'''
red_fighters = data['R_fighter'].unique().tolist()
blue_fighters = data['B_fighter'].unique().tolist() 
all_fighters = red_fighters + blue_fighters
unique_fighters = []

f = open("nodes.csv", "w")
writer = csv.writer(f)
writer.writerow(['Fighter'])
for i in all_fighters:
    if i not in unique_fighters:
        unique_fighters.append(i)
        writer.writerow([i])
f.close()

In [14]:
edge_list = []
f = open("edges.csv", "w")
writer = csv.writer(f)
writer.writerow(['Source', 'Target'])
for index, row in data.iterrows():
    edge_list.append((row['R_fighter'], row['B_fighter']))
    writer.writerow(row[['R_fighter', 'B_fighter']])
f.close()

In [15]:
G = nx.Graph()
G.add_nodes_from(unique_fighters)
G.add_edges_from(edge_list)
len(unique_fighters)

2139

In [16]:
nx.write_gml(G, "undirected.gml")

# Generate Directed Graph

In [17]:
def generate_graph_from_data(data):
    G_directed = nx.DiGraph()

    G_directed.add_nodes_from(unique_fighters)

    directed_edge_list = []
    for index, row in data.iterrows():
        red_fighter = row['R_fighter']
        red_fighter_stance = row['R_Stance']

        blue_fighter = row['B_fighter']
        blue_fighter_stance = row['B_Stance']

        weight_class = row['weight_class']
        title_bout = row['title_bout']

        referee = row['Referee']
        if pd.isnull(row['Referee']):
            referee = "N/A"

        # Iteratievley check if an edge has already been added to the graph, add 1 to
        # the repeat bout weight
        if row['Winner'] == 'Blue':
            if G_directed.has_edge(red_fighter, blue_fighter):
                G_directed[red_fighter][blue_fighter]['repeated_bout_weight'] += 1
            else:
                G_directed.add_edge(red_fighter, 
                                    blue_fighter, 
                                    repeated_bout_weight=1, 
                                    weight_class=weight_class, 
                                    title_bout=title_bout,
                                    referee=referee
                                   )
        elif row['Winner'] == 'Red':
            if G_directed.has_edge(blue_fighter, red_fighter):
                G_directed[blue_fighter][red_fighter]['repeated_bout_weight'] += 1
            else:
                G_directed.add_edge(blue_fighter, 
                                    red_fighter, 
                                    repeated_bout_weight=1, 
                                    weight_class=weight_class, 
                                    title_bout=title_bout,
                                    referee=referee
                                   )
        else:
            # Its a draw, so add both edges between the fighters
            if G_directed.has_edge(red_fighter, blue_fighter):
                G_directed[red_fighter][blue_fighter]['repeated_bout_weight'] += 1
            else:
                G_directed.add_edge(red_fighter, 
                                    blue_fighter, 
                                    repeated_bout_weight=1, 
                                    weight_class=weight_class, 
                                    title_bout=title_bout,
                                    referee=referee
                                   )
            if G_directed.has_edge(blue_fighter, red_fighter):
                G_directed[blue_fighter][red_fighter]['repeated_bout_weight'] += 1
            else:
                G_directed.add_edge(blue_fighter, 
                                    red_fighter, 
                                    repeated_bout_weight=1, 
                                    weight_class=weight_class, 
                                    title_bout=title_bout,
                                    referee=referee
                                   )
    return G_directed
    

In [18]:
# A multigraph: a directed graph with multiple edges (multiple fights)
G_directed = generate_graph_from_data(data)

In [19]:
# export the directed edge list
nx.write_gml(G_directed, "directed.gml")